## Installation and Imports

In [ ]:
!pip install ray[rllib] torch
!pip install tensorflow_probability
!pip install wandb

In [ ]:
# Check if GCC is installed
!gcc --version

In [ ]:
# Install GCC if absent
!sudo apt update
!sudo apt install build-essential -y

In [ ]:
# # Setup Mujoco for gym
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

# !pip install gym

# !pip install free-mujoco-py

# import mujoco_py
# import gym

In [1]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym

In [2]:
# Test that mujoco for gym is available
gym.make("HalfCheetah-v3")

<TimeLimit<HalfCheetahEnv<HalfCheetah-v3>>>

## Configure and Run

In [3]:
config = (
    SACConfig()
    .environment(
        env="HalfCheetah-v3",
        normalize_actions=True
    )
    .training(
        q_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        policy_model_config={
            "fcnet_activation": "relu",
            "fcnet_hiddens": [256, 256]
        },
        tau=0.005,
        target_entropy="auto",
        n_step=1,  # num of SGD steps per batch of data in training step
        train_batch_size=256,
        target_network_update_freq=1,
        replay_buffer_config={"type":"MultiAgentPrioritizedReplayBuffer"},
        num_steps_sampled_before_learning_starts=10_000,
        optimization_config={
          "actor_learning_rate": 0.0003,
          "critic_learning_rate": 0.0003,
          "entropy_learning_rate": 0.0003,
        },
        clip_actions=False
    )
    .rollouts(
        num_rollout_workers=3,
        rollout_fragment_length=1,
    )
    .resources(num_gpus=0)
    .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .reporting(
        min_sample_timesteps_per_iteration=1000,
        metrics_num_episodes_for_smoothing=5
    )
    .framework(framework="torch")
)

In [4]:
wandb_init = dict(
    save_code=True,
    config={
        "env": "HalfCheetah-v3",
        
        "actor_learning_rate": 0.0003,
        "critic_learning_rate": 0.0003,
        "entropy_learning_rate": 0.0003,
        "framework": "torch",
        
        "num_rollout_workers": 3,
        "num_gpu": 0,
        "metrics_num_episodes_for_smoothing": 5
    },
    tags=["local"],
    notes="Test to inspect scaling on Vast.ai",
    name="HalfCheetah_local"
    # job_type=D
    # monitor_gym=
)

In [5]:
tuner = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        name="HalfCheetah_local",
        stop={"training_iteration": 3_000, "episode_reward_mean": 150},
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
        callbacks=[
                WandbLoggerCallback(project="HalfCheetah", 
                                    api_key="392c8a47eb0658eb5c71190757a69110e2140f4a",
                                    save_checkpoints=True, 
                                    **wandb_init)
            ],
        local_dir="./results"
        ),
    param_space=config
)

results = tuner.fit()

2023-02-10 13:05:53,494	INFO worker.py:1538 -- Started a local Ray instance.


wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(SAC pid=14767) 2023-02-10 13:05:59,419	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=14767) 2023-02-10 13:05:59,642	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) [2023-02-10 13:06:03,390 E 14506 14551] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-10_13-05-51_350412_13852 is over 95% full, available space: 1204518912; capacity: 31845081088. Object creation will fail if spilling is required.


(SAC pid=14767) 2023-02-10 13:06:10,815	INFO trainable.py:172 -- Trainable.setup took 11.177 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(SAC pid=14767) 2023-02-10 13:06:10,815	WARNING util.py:66 -- Install gputil for GPU system monitoring.
(SAC pid=14767) 2023-02-10 13:06:11,087	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(raylet) [2023-02-10 13:06:13,402 E 14506 14551] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-10_13-05-51_350412_13852 is over 95% full, available space: 1204424704; capacity: 31845081088. Object creation will fail if spilling is required.


(raylet) [2023-02-10 13:06:23,409 E 14506 14551] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-10_13-05-51_350412_13852 is over 95% full, available space: 1204297728; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-10 13:06:33,414 E 14506 14551] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-10_13-05-51_350412_13852 is over 95% full, available space: 1204154368; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-10 13:06:43,420 E 14506 14551] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-10_13-05-51_350412_13852 is over 95% full, available space: 1204060160; capacity: 31845081088. Object creation will fail if spilling is required.
(SAC pid=14767) 2023-02-10 13:06:44,804	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!